In [1]:
import degirum as dg
import requests
import json
import os
import tempfile
from contextlib import contextmanager
from dg_coco_utils import *
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
import numpy as np

In [2]:
@contextmanager
def annotations_accessor(url:str):
    """Annotation accessor context manager: downloads annotations into local temporary file and returns its name
    
    - `url`: annotations url 
    """
    r = requests.get(url)
    r.raise_for_status()
    fd, fname = tempfile.mkstemp()
    try:
        with os.fdopen(fd, 'w') as f:
            json.dump(r.json(), f)
        yield fname
    finally:
        os.unlink(fname)

In [3]:
json_path = '/home/degirum/yolov8_test/yolov8n_relu6_quant.json'
with open(json_path) as config:
    config_options=json.load(config)
# ai_server_address='localhost'
models=dg.connect(dg.LOCAL, json_path)
ground_truth_annotations_path="http://100.102.102.108:8084/coco/val/labels.json"
image_folder_path="http://100.102.102.108:8084/coco/val/data/"
pred_json='yolov5s_edgetpu_predictions.json'
# print_frequency=50

In [4]:
model_name=config_options['MODEL_PARAMETERS'][0]['ModelPath']
eval_model=models.load_model(model_name)
# eval_model.output_confidence_threshold=config_options['POST_PROCESS'][0]['OutputConfThreshold']
# eval_model.output_nms_threshold=config_options['POST_PROCESS'][0]['OutputNMSThreshold']
# eval_model.output_max_detections=config_options['POST_PROCESS'][0]['MaxDetections']
# eval_model.output_max_detections_per_class=config_options['POST_PROCESS'][0]['MaxDetectionsPerClass']
# eval_model.output_max_classes_per_detection=config_options['POST_PROCESS'][0]['MaxClassesPerDetection']
# eval_model.output_use_regular_nms=config_options['POST_PROCESS'][0]['UseRegularNMS']
# eval_model.input_resize_method=config_options['PRE_PROCESS'][0]['InputResizeMethod']
# eval_model.input_pad_method=config_options['PRE_PROCESS'][0]['InputPadMethod']
# eval_model.image_backend=config_options['PRE_PROCESS'][0]['ImageBackend']
class_map=config_options['POST_PROCESS'][0]['ClassMap']  

In [5]:
jdict=[]
with annotations_accessor(ground_truth_annotations_path) as annotations_file:
    anno = COCO(annotations_file)
num_images=len(anno.dataset['images'])
files_dict=anno.dataset['images'][0:num_images]

for image_number in tqdm(range(0,num_images)):
#     if image_number%print_frequency==print_frequency-1:
#         print(image_number+1)
    image_id=files_dict[image_number]['id']
    path=image_folder_path + files_dict[image_number]['file_name']
#     print(path)
    predictions=eval_model(path)
    save_results_coco_json(predictions.results,jdict,image_id,class_map)
with open(pred_json, 'w') as f:
    json.dump(jdict, f)

pred = anno.loadRes(pred_json)
eval_obj = COCOeval(anno, pred, 'bbox')
eval_obj.params.imgIds = sorted([file['id'] for file in files_dict])  # image IDs to evaluate
eval_obj.evaluate()
eval_obj.accumulate()
eval_obj.summarize()
map_all, map50 = eval_obj.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)

loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


100%|████████████████████████████████████████████████████████████████| 5000/5000 [04:09<00:00, 20.02it/s]


Loading and preparing results...
DONE (t=3.37s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=27.26s).
Accumulating evaluation results...
DONE (t=10.45s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.331
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.483
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.367
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.466
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.297
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.529
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=

In [6]:
eval_obj.stats

array([0.33059591, 0.48300294, 0.35478269, 0.14950367, 0.36672598,
       0.46578683, 0.29656854, 0.50201345, 0.52927319, 0.3096648 ,
       0.59198686, 0.71686478])

In [7]:
# config_options['Results']